## Парафраз

Загрузим [данные](http://paraphraser.ru/download/get?file_id=1).

In [1]:
!unzip paraphraser.zip

Archive:  paraphraser.zip
  inflating: LICENSE                 
  inflating: corpus.xml              
  inflating: paraphrases.xml         


In [2]:
with open('paraphrases.xml') as f:
    data = f.read()

Как это выглядит:

In [3]:
!tail paraphrases.xml

      <value name="id">25577</value>
      <value name="id_1">34584</value>
      <value name="id_2">34722</value>
      <value name="text_1">9 мая метрополитен Петербурга будет работать круглосуточно</value>
      <value name="text_2">Мартынов: комендантский час в Донецке 9 мая будет отменен</value>
      <value name="jaccard">0.46188021535170065</value>
      <value name="class">-1</value>
    </paraphrase>
  </corpus>
</data>


### Создаём датасет для обучения

In [4]:
import pandas as pd

In [5]:
from xml.etree import ElementTree

In [6]:
root = ElementTree.XML(data)

In [7]:
par_data = {'text1': [], 'text2': [], 'class': []}
for par in root[1]:
    par_data['text1'].append(par[3].text)
    par_data['text2'].append(par[4].text)
    par_data['class'].append(int(par[6].text))

In [8]:
parphrase_df = pd.DataFrame(par_data)
parphrase_df.head(10)

,class,text1,text2
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...
5,1,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...
6,-1,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...
7,0,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...
8,-1,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...
9,-1,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...


А теперь давайте:
* векторизуем каждый документ
* посчитаем расстояние между каждой парой
* на этом обучим классификатор

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tfidf_vec = TfidfVectorizer()

In [11]:
text1_vecs = tfidf_vec.fit_transform(parphrase_df.text1)

In [12]:
text2_vecs = tfidf_vec.transform(parphrase_df.text2)

In [21]:
from sklearn.decomposition import TruncatedSVD

In [15]:
from gensim import similarities

In [20]:
index = similarities.MatrixSimilarity(text1_vecs)

ValueError: not enough values to unpack (expected 2, got 1)